# --- Day 14: Extended Polymerization ---

The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:
```
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
```
The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like AB -> C means that when elements A and B are immediately adjacent, element C should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template NNCB, the first step simultaneously considers all three pairs:

The first pair (NN) matches the rule NN -> C, so element C is inserted between the first N and the second N.
The second pair (NC) matches the rule NC -> B, so element B is inserted between the N and the C.
The third pair (CB) matches the rule CB -> H, so element H is inserted between the C and the B.
Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes NCNBCHB.

Here are the results of a few steps using the above rules:
```
Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
```
This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, B occurs 1749 times, C occurs 298 times, H occurs 161 times, and N occurs 865 times; taking the quantity of the most common element (B, 1749) and subtracting the quantity of the least common element (H, 161) produces 1749 - 161 = 1588.

### Part One

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

### Part Two

The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of 40 steps should do it.

In the above example, the most common element is B (occurring `2192039569602` times) and the least common element is H (occurring `3849876073` times); subtracting these produces `2188189693529`.

Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

### Inputs and Imports

In [1]:
input_file_list, sample_input_file_list = [], []

day = '14'

with open(f'Inputs\\day_{day}.txt', 'r') as input_file: 
    for line in input_file.readlines():
        input_file_list.append(line.rstrip('\n'))
with open(f'C:Inputs\\day_{day}_sample.txt', 'r') as input_file: 
    for line in input_file.readlines():
        sample_input_file_list.append(line.rstrip('\n'))        

########################        
# Part One Sample Answer: 1588
########################
# Part Two Sample Answer: 2188189693529
########################

In [13]:
def get_template_and_rules(raw_input: list) -> "tuple (str, dict)":
    '''
    Parse the raw_input and output 
        - a template (str) e.g. 'AB'
        - a set of rules (dict) in the format {'AB': 'C'}
    '''
    template  = raw_input[:raw_input.index('')][0]
    
    # parsing the rules into a dic
    raw_rules = raw_input[raw_input.index('')+1:]
    rules = {}
    for rule in raw_rules:
        k, v = rule.split(' -> ')
        rules[k] = v 
        
    return template, rules


def create_empty_rules_counter(rules: dict) -> "dict {str: 0}" :
    '''
    Creating a dict with each key being the rule key with a counter value of zero 
        - Format (str: int)
        - Example {'AB': 0}
    This will be used to count each time a pair is seen within a template
    '''
    empty_rules_counter = {key:0 for key in rules}
    
    return empty_rules_counter


def get_initial_state(rules: dict,
                      template: str) -> "dict {str: int}":
    '''
    Find the initial pairs from within the template 
    e.g. 'ABCD' would create these pairs: 'AB'/'BC'/'CD'
    Then increase the count of these pairs to get an 'initial state of the counter dictionary'
    '''
    rules_counter = create_empty_rules_counter(rules)
    for idx in range(len(template)-1):        
        pair = f'{template[idx]}{template[idx+1]}'
        rules_counter[pair] += 1
        
    return rules_counter    

def single_step(rules: dict, 
                rules_counter: dict ) -> "dict {str: int}":  
    '''
    Outputs a new rules counter with all the values of each pair after a single step
    
    - Each pair will create 2 new pairs, 
    - So if the rule was 'AB' -> C then we would see the folling new pairs created: 'AC' / 'CB'
    - We then create a new rules counter and increase the value for the new pairs by the value of the original pair
    - So if we started with 'AB':
        - Template = 'AB'
        - Rules Counter = {'AB':1, 'AC':0, 'CB':0} 
    - We would finish with this:
        - Template = 'ACB'
        - Rules Counter = {'AB':0, 'AC':1, 'CB':1} 
    '''
    new_rules_counter = create_empty_rules_counter(rules)

    for key, value in rules.items():       
        new_pair_1 = key[0]+value
        new_pair_2 = value+key[1]
        new_rules_counter[new_pair_1] += rules_counter[key]
        new_rules_counter[new_pair_2] += rules_counter[key]
        
    return new_rules_counter    

def get_letter_frequencies(rules: dict, 
                           rules_counter: dict,
                           template: str) -> "list(int)":
    '''
    - Outputs a list of ints
    - We are looking for the freq of the most common letter within the template minus the least common
    - to calculate the volume of each letter in the final template you want to follow this pattern:
        - if key: counter = {'NC': 1}
        - then add 1 to each letter: {'N': 1, C: 1}
        - once you have gone through all the rules, add one to the first and final letter from the original template
        - e.g. if the original template is 'NNCB' you would add an additional 1 to both 'N' and 'B'
    - then finallaly you want to divide each counter value by 2 due to the repetition you'll see in the pairings
    - adding the addition count to the first and last letters accounts for each of these letters only appearing in one pair 
    - as they bookend the template string
    '''
    # create a distinct list of all possible letters from the rules
    letters = list(set([x[0] for x in rules]))
    letters_counter = {letter:0 for letter in letters}

    # # for each letter in each rule key add the value to it's corresponding letter record in the 'letters_counter' dict
    for key, value in rules_counter.items():
        letters_counter[key[0]] += value
        letters_counter[key[1]] += value

    # increase the counter by 1 for the first and last character within the template
    for i in [template[0], template[-1]]: letters_counter[i] += 1

    # dividing all letter values by 2 and adding to a 'letter_frequencies' list e.g. [2, 6, 9, 1] 
    letter_frequencies = [int(letters_counter[key]/2) for key in letters_counter]
    
    return letter_frequencies

def step_through(input_file    :list,
                 sample_input  :list,
                 sample_flag   :bool,
                 steps         :int,
                 steps_to_print:list = []
                 ):
    '''
    - Iterate over the volume of steps passed into the paramter
    - Prints the answer (most frequent letter volume - least frequent letter volume)
    - Also prints the size of the final template
    '''
    # use the main input or the sample input depending on flag parmeter
    raw_input = input_file if sample_flag is not True else sample_input

    # parse raw input into the following variables
    template, rules = get_template_and_rules(raw_input = raw_input)

    # update counter_rules with the initial state
    rules_counter = get_initial_state(rules = rules, template= template)
    
    print(f'Initial Polymer Template: {template}')

    for step in range(steps):
        new_rules_counter = single_step(rules, rules_counter)
        letter_frequencies = get_letter_frequencies(rules, new_rules_counter, template) 
        answer = max(letter_frequencies) - min(letter_frequencies)
        if steps_to_print != []:
            if step+1 in steps_to_print:
                print(f'After step {step+1}: answer = {answer} / closing template length = {sum(letter_frequencies)}')
        else: print(f'After step {step+1}: answer = {answer} / closing template length = {sum(letter_frequencies)}')
        rules_counter = new_rules_counter
        
step_through(input_file     = input_file_list,
             sample_input   = sample_input_file_list,
             sample_flag    = False,
             steps          = 1000,
             steps_to_print = [1000]
            )

Initial Polymer Template: CFFPOHBCVVNPHCNBKVNV
After step 1000: answer = 31498647363818065264860140617973448221515313378838787391597962193044305834566204156646616413011835624941640859197598933500414932931653150589336387788474857195401121413834065932528963233494342656206973762517720118661332822985168101325934161864030086936903703662714120547732017437132438831442157002293248 / closing template length = 203586635365390798117881920014759045437366905343574074044361541755460103181442787990222846344923206061050897435885580727819431797207810310465716152534820318670743577013154592481260451643992446620575896665537020001836196639367475172749339127989288618530927646749603099643364672981290644903980582003802112


### Without Commentary

In [ ]:
def get_template_and_rules(raw_input: list) -> "tuple (str, dict)":
    template  = raw_input[:raw_input.index('')][0]
    raw_rules = raw_input[raw_input.index('')+1:]
    rules = {}
    for rule in raw_rules:
        k, v = rule.split(' -> ')
        rules[k] = v 
    return template, rules

def create_empty_rules_counter(rules: dict) -> "dict {str: 0}" :
    empty_rules_counter = {key:0 for key in rules}
    return empty_rules_counter

def get_initial_state(rules: dict, template: str) -> "dict {str: int}":
    rules_counter = create_empty_rules_counter(rules)
    for idx in range(len(template)-1):        
        pair = f'{template[idx]}{template[idx+1]}'
        rules_counter[pair] += 1     
    return rules_counter    

def single_step(rules: dict, rules_counter: dict ) -> "dict {str: int}":  
    new_rules_counter = create_empty_rules_counter(rules)
    for key, value in rules.items():       
        new_rules_counter[key[0]+value] += rules_counter[key]
        new_rules_counter[value+key[1]] += rules_counter[key]        
    return new_rules_counter    

def get_letter_frequencies(rules: dict, rules_counter: dict, template: str) -> "list(int)":
    letters = list(set([x[0] for x in rules]))
    letters_counter = {letter:0 for letter in letters}
    for key, value in rules_counter.items():
        letters_counter[key[0]] += value
        letters_counter[key[1]] += value
    for i in [template[0], template[-1]]: letters_counter[i] += 1
    return [int(letters_counter[key]/2) for key in letters_counter] # letter_frequencies

def step_through(input_file:list, sample_input:list, sample_flag:bool, steps:int, steps_to_print:list = []):
    raw_input = input_file if sample_flag is not True else sample_input
    template, rules = get_template_and_rules(raw_input = raw_input)
    rules_counter = get_initial_state(rules = rules, template= template)
    print(f'Initial Polymer Template: {template}')
    for step in range(steps):
        new_rules_counter = single_step(rules, rules_counter)
        letter_frequencies = get_letter_frequencies(rules, new_rules_counter, template) 
        answer = max(letter_frequencies) - min(letter_frequencies)
        if steps_to_print:
            if step+1 in steps_to_print:
                print(f'After step {step+1}: answer = {answer} / closing template length = {sum(letter_frequencies)}')
        else: print(f'After step {step+1}: answer = {answer} / closing template length = {sum(letter_frequencies)}')
        rules_counter = new_rules_counter
        
step_through(input_file     = input_file_list,
             sample_input   = sample_input_file_list,
             sample_flag    = False,
             steps          = 40,
             steps_to_print = [10,40]
            )